In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import urllib.request, json, re
import pprint as pp
import tensorflow as tf
import re

from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import time
import datetime


import random
np.random.seed(1)

# Klassformat för lagring och serialisering
    

In [25]:
class DocList:
    data = []
    def __init__(self):
        self.name='MotionList'
        self.data = []
    def reprJSON(self):
        return dict(name=self.name, data=self.data) 
    
class Motion:
    def __init__(self):
        self.text=''
        self.title=''
        self.subtype=''
        self.meta = {}
    def reprJSON(self):
        return dict(text=self.text, subtype=self.subtype, title=self.title, meta=self.meta) 
    
class DocSerializer(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj,'reprJSON'):
            return obj.reprJSON()
        else:
            return json.JSONEncoder.default(self, obj)



## Import och skapande av datafil
denna koden ska bara köras en gång vid inläsning av data från riksdagen

In [34]:

def import_web_data(datasize=3):

    #http://data.riksdagen.se/dokumentlista/?doktyp=mot
    #http://data.riksdagen.se/dokument/H502225.text (exempeldok.)
    #"http://data.riksdagen.se/anforandelista/?rm=&anftyp=Nej&d=&ts=&parti=s&iid=0218878014918&sz="+str(datasize)+"&utformat=json"
    #http://data.riksdagen.se/anforandelista/?rm=&anftyp=&d=&ts=&parti=&iid=&sz=10&utformat=xml
    motioner = DocList()
    with urllib.request.urlopen("http://data.riksdagen.se/dokumentlista/?doktyp=mot&sz="+str(datasize)+"&utformat=json") as url:
        rawdata = json.loads(url.read().decode())
        motionslista = rawdata['dokumentlista']['dokument']
        for meta in motionslista:
            motion = Motion()
            with urllib.request.urlopen('http:' + meta['dokument_url_text']) as text:
                motion.text = re.sub('\n','',text.read().decode())
                #motion.meta = meta (save space appr 30%)
                motion.title = meta['titel']
                motion.subtype = meta['subtyp']
            motioner.data.append(motion)

     
    toJson = json.dumps(motioner.reprJSON(), cls=DocSerializer,  sort_keys=True, indent=4)
    with open('motioner.txt','w',encoding='utf-8') as f:
        
        f.write(toJson) #json.dumps(motions.reprJSON()))#json.dumps(motions, sort_keys=True, indent=4))
        
    return toJson, motioner

#datasize är antal dokument vi behöver förmodligen ett hundratal iaf.
#data_ex, motionList = import_web_data(datasize=500)
print("totalt antal dokument: " + str(len(motionList.data)))



totalt antal dokument: 200
